In [1]:
from pdg_js.build_pdg import get_data_flow
from pdg_js.js_operators import get_node_computed_value
import config as config 


def check_page_compliance(r, scopes:set, sensi_apis:set, flag_navigate:bool, url:str):
    scopes, sensi_apis, flag_navigate, url = find_page_methods_node(r, scopes, sensi_apis, flag_navigate, url)
    if len(scopes) > 1 or len(sensi_apis) > 1:
        print("存在一揽子授权行为, 相关权限/api包括: ", end=' ')
        print(scopes, end=' ')
        print(sensi_apis)
    elif len(scopes) == 1 and len(sensi_apis) == 1:
        print("存在启动即弹窗授权, 相关权限/api包括: ", end=' ')
        print(scopes, end=' ')
        print(sensi_apis)
    if flag_navigate:
        print("存在强制索权行为, 强制索权页面为: ", end=' ')
        print(url)


def find_page_methods_node(r, scopes:set, sensi_apis:set, flag_navigate:bool, url:str):
    for child in r.children:
        if child.name == "ExpressionStatement":
            if len(child.children) > 0 \
                    and child.children[0].name == "CallExpression" \
                    and child.children[0].children[0].attributes["name"] == "Page":
                # found page expression
                for method_node in child.children[0].children[1].children:  # method_node: Property
                    if method_node.attributes["value"]["type"] == "FunctionExpression":
                        # handle node
                        method_name = method_node.children[0].attributes['name']
                        print(f"[page method] got page method, method name: {method_name}")
                        if method_name == "onLoad":
                            scopes, sensi_apis, flag_navigate, url = check_compliance(method_node, scopes, sensi_apis, flag_navigate, url)
    return scopes, sensi_apis, flag_navigate, url


def check_compliance(method_node, scopes:set, sensi_apis:set, flag_navigate:bool, url:str):
    for child in method_node.children:
        if child.name in ('CallExpression', 'TaggedTemplateExpression'):
            if len(child.children) > 0 and child.children[0].body in ('callee', 'tag'):
                callee = child.children[0]
                call_expr_value = get_node_computed_value(callee)
                call_expr_value_all = get_node_computed_value(child)
                child.set_value(call_expr_value_all)
                if call_expr_value == 'wx.authorize':
                    scopes = check_authorize_scopes(child, scopes)
                    flag_navigate, url = check_fail_callback(child, flag_navigate, url)
                elif call_expr_value in config.SENSITIVE_API:
                    sensi_apis.add(call_expr_value)
        scopes, sensi_apis, flag_navigate, url = check_compliance(child, scopes, sensi_apis, flag_navigate, url)
    return scopes, sensi_apis, flag_navigate, url


def check_authorize_scopes(child, scopes: set()):
    obj_exp = child.children[1]
    for prop in obj_exp.children:
        if get_node_computed_value(prop.children[0]) == 'scope':
            scope = get_node_computed_value(prop.children[1])
            scopes.add(scope)
    return  scopes


def check_fail_callback(child, flag_navigate, url):
    obj_exp = child.children[1]
    for prop in obj_exp.children:
        if get_node_computed_value(prop.children[0]) == 'fail':
            flag_navigate, url = check_if_navigate(prop, flag_navigate, url)
    return flag_navigate, url


def check_if_navigate(child, flag_navigate, url):
    for child in child.children:
        if child.name in ('CallExpression', 'TaggedTemplateExpression'):
           if len(child.children) > 0 and child.children[0].body in ('callee', 'tag'):
                callee = child.children[0]
                call_expr_value = get_node_computed_value(callee)
                if call_expr_value == 'wx.navigateTo':
                    obj_exp = child.children[1]
                    for prop in obj_exp.children:
                        flag_navigate = True
                        url = get_node_computed_value(prop.children[1])
        flag_navigate, url = check_if_navigate(child, flag_navigate, url)
    return flag_navigate, url


##### Case1: 启动即弹窗授权

In [2]:
pdg_node = get_data_flow(input_file="testcase/case1.js", benchmarks={}, save_path_pdg="testcase/case1/pdg", \
                         save_path_ast="testcase/case1/ast", save_path_cfg="testcase/case1/cfg")
check_page_compliance(pdg_node, scopes=set(), sensi_apis=set(), flag_navigate = False, url='')

CURRENT STATE Successfully got Esprima AST in 0.10683443769812584s
CURRENT STATE Successfully produced the AST in 0.0016973093152046204s
CURRENT STATE Successfully produced the CFG in 0.05273934919387102s
CURRENT STATE Successfully produced the PDG in 0.050686669535934925s
[page method] got page method, method name: onLoad
存在启动即弹窗授权, 相关权限/api包括:  {'scope.userLocation'} {'wx.getLocation'}


##### Case2: 一揽子授权

In [3]:
pdg_node = get_data_flow(input_file="testcase/case2.js", benchmarks={}, save_path_pdg="testcase/case2/pdg")
check_page_compliance(pdg_node, scopes=set(), sensi_apis=set(), flag_navigate = False, url='')

CURRENT STATE Successfully got Esprima AST in 0.1086621405556798s
CURRENT STATE Successfully produced the AST in 0.0013032695278525352s
CURRENT STATE Successfully produced the CFG in 0.00023468676954507828s
CURRENT STATE Successfully produced the PDG in 0.0011695045977830887s
[page method] got page method, method name: onLoad
存在一揽子授权行为, 相关权限/api包括:  {'scope.bluetooth', 'scope.camera'} {'wx.openBluetoothAdapter', 'wx.createCameraContext'}


##### Case3: 强制索权

In [4]:
pdg_node = get_data_flow(input_file="testcase/case3.js", benchmarks={}, save_path_pdg="testcase/case3/pdg")
check_page_compliance(pdg_node, scopes=set(), sensi_apis=set(), flag_navigate = False, url='')

CURRENT STATE Successfully got Esprima AST in 0.10671781469136477s
CURRENT STATE Successfully produced the AST in 0.0011067120358347893s
CURRENT STATE Successfully produced the CFG in 0.0002650078386068344s
CURRENT STATE Successfully produced the PDG in 0.0017657224088907242s
[page method] got page method, method name: onLoad
存在启动即弹窗授权, 相关权限/api包括:  {'scope.userLocation'} {'wx.getLocation'}
存在强制索权行为, 强制索权页面为:  pages/authorize/authorize
